In [1]:
%matplotlib inline
import nuwinter
import matplotlib.pyplot as plt
from nuztf.neutrino_scanner import NeutrinoScanner
from astropy import units as u
from nuwinter.avro import load_avro
from nuwinter.plot import ann_fields, generate_single_page
from nuwinter.utils import deduplicate_df
from nuwinter.paths import get_pdf_path
from pathlib import Path
import pandas as pd
import json
from astropy import units as u
from astropy.coordinates import SkyCoord
from matplotlib.colors import Normalize
from matplotlib.ticker import MultipleLocator
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from tqdm import tqdm
from nuztf.skymap_scanner import SkymapScanner
from nuztf.ampel import ampel_api_catalog
from nuztf.api import api_name
from nuwinter.ztf import add_ztf_candidates, ZTF_NAME_KEY, ZTF_HIST_KEY

/opt/anaconda3/envs/nuwinter/lib/python3.11/site-packages/ligo/lw/lsctables.py:89: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal


In [2]:
superstack = False
crossmatch = True

base_dir = Path.home().joinpath(
    "Data/observing/winter_superstack" if superstack else "Data/observing/winter"
)

# name = "IC240412A"

name = "S250206dm"

prob_threshold = 0.95
n_days = 14

In [3]:
base_outpath = str(get_pdf_path(name))
if superstack:
    base_outpath = str(base_outpath).replace("S250206dm", "S250206dm_superstack")
Path(base_outpath).parent.mkdir(parents=True, exist_ok=True)
base_outpath

'/Users/rdstein/Data/nuwinter/S250206dm/winter_candidate_pdf/combined.pdf'

In [4]:
neutrino = "IC" in name
nu = NeutrinoScanner(name) if neutrino else SkymapScanner(
    event=name,
    prob_threshold=prob_threshold,
    n_days=n_days
)

100%|███████████████████████████████████| 1050/1050 [00:00<00:00, 392830.18it/s]


In [5]:
ztf_initial_cache = nu.get_initial_cache_path()

download_ztf = False

if not (ztf_initial_cache.exists()):
    download_ztf = True

if ztf_initial_cache.exists():
    if not ztf_initial_cache.stat().st_size > 2:
        download_ztf = True

if download_ztf:
    print(f"No file found at {ztf_initial_cache}")
    nu.scan_area(t_max=nu.t_min + n_days)

ztf_initial_cache

PosixPath('/Users/rdstein/Data/nuztf/cache/candidates/S250206dm/rev7/0.95/initial_stage.json')

In [6]:
nights = [str(nu.t_min + (i - 1) * u.day).split("T")[0].replace("-", "") for i in range(n_days + 1)]

all_res = []

for night in sorted(nights):
    
    avro_dir = base_dir / f"{night}/avro"

    print(f"Looking for avros in {avro_dir}")
    
    avro_files = list(avro_dir.glob("*.avro"))
    
    for path in avro_files:
        all_res.append(load_avro(path))

if len(all_res) == 0:
    raise FileNotFoundError(f"No avro files found for any night, please download these first using the download notebook!")

res_winter = pd.concat(all_res)
res_winter[ZTF_NAME_KEY] = res_winter[ZTF_NAME_KEY].replace({"nan": None})

Looking for avros in /Users/rdstein/Data/observing/winter/20250205/avro
Looking for avros in /Users/rdstein/Data/observing/winter/20250206/avro
Looking for avros in /Users/rdstein/Data/observing/winter/20250207/avro
Looking for avros in /Users/rdstein/Data/observing/winter/20250208/avro
Looking for avros in /Users/rdstein/Data/observing/winter/20250209/avro
Looking for avros in /Users/rdstein/Data/observing/winter/20250210/avro
Looking for avros in /Users/rdstein/Data/observing/winter/20250211/avro
Looking for avros in /Users/rdstein/Data/observing/winter/20250212/avro
Looking for avros in /Users/rdstein/Data/observing/winter/20250213/avro
Looking for avros in /Users/rdstein/Data/observing/winter/20250214/avro
Looking for avros in /Users/rdstein/Data/observing/winter/20250215/avro
Looking for avros in /Users/rdstein/Data/observing/winter/20250216/avro
Looking for avros in /Users/rdstein/Data/observing/winter/20250217/avro
Looking for avros in /Users/rdstein/Data/observing/winter/202502

/var/folders/y5/nrs6cfxd0d99x77tn7tlscsm0000gr/T/ipykernel_9951/2524381070.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res_winter = pd.concat(all_res)


In [7]:
# print(f"Deduplicating {len(res_winter)} alerts")
# res_winter = deduplicate_df(res_winter)

In [8]:
# Select only those in contour
in_contour = []

for _, row in res_winter.iterrows():
    in_contour.append(nu.in_contour(row["ra"], row["dec"]))

print(f"{np.sum(in_contour)} / {len(in_contour)} candidates in contour")

print(f"{len(set(res_winter['objectid'][in_contour]))} / {len(set(res_winter['objectid']))} sources in contour")

res_winter = res_winter[in_contour]

62675 / 62991 candidates in contour
57670 / 57986 sources in contour


In [9]:
res_winter

,objectid,cutout_science,cutout_template,cutout_difference,prv_candidates,candid,deprecated,jd,fid,exptime,...,tmkmag3,tmobjectid3,distgaia,plxgaia,ruwegaia,distgaiabright,plxgaiabright,ruwegaiabright,ztfname,distztf
0,WNTR25aacys,b'\x1f\x8b\x08\x00\xd0\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd0\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd0\xaa\xb6g\x02\xff\xec{i4...,candid deprecated jd fid expt...,37036,False,2.460715e+06,2,960.0,...,14.864,02465534+5447396,6.947795,0.462795,0.970511,75.220940,161.797745,1.008506,None,NaN
1,WNTR25aacyt,b'\x1f\x8b\x08\x00\xd1\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd1\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd1\xaa\xb6g\x02\xff\xec\x9...,candid deprecated jd fid expt...,37037,False,2.460715e+06,2,960.0,...,13.901,02455735+5448057,0.769464,17.789389,1.012096,17.490801,64.846901,0.981179,ZTF18acmgeko,0.231566
2,WNTR25aacyu,b'\x1f\x8b\x08\x00\xd2\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd2\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd2\xaa\xb6g\x02\xff\xecweP...,candid deprecated jd fid expt...,37038,False,2.460715e+06,2,960.0,...,15.024,02443253+5447502,4.838482,212.397964,0.984985,4.838482,212.397964,0.984985,None,NaN
3,WNTR25aacyv,b'\x1f\x8b\x08\x00\xd3\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd3\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd3\xaa\xb6g\x02\xff\xec\x9...,candid deprecated jd fid expt...,37039,False,2.460715e+06,2,960.0,...,NaN,nan,4.637819,0.428713,1.017993,NaN,NaN,NaN,None,NaN
4,WNTR25aacyw,b'\x1f\x8b\x08\x00\xd4\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd4\xaa\xb6g\x02\xff\xecwy4...,b'\x1f\x8b\x08\x00\xd4\xaa\xb6g\x02\xff\xec\x9...,candid deprecated jd fid expt...,37040,False,2.460715e+06,2,960.0,...,16.421,02464481+5447190,11.883331,0.293812,1.056229,NaN,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WNTR25acewv,b'\x1f\x8b\x08\x00P_\xb7g\x02\xff\xec\xbai4W\x...,b'\x1f\x8b\x08\x00P_\xb7g\x02\xff\xec\x99yTN\x...,b'\x1f\x8b\x08\x00P_\xb7g\x02\xff\xec\x97\xf97...,candid deprecated jd fid expt...,75403,False,2.460718e+06,2,960.0,...,NaN,nan,23.549694,32.031876,1.009527,86.424377,37.304741,0.994731,None,NaN
196,WNTR25aceww,b'\x1f\x8b\x08\x00Q_\xb7g\x02\xff\xec\x97i4\xd...,b'\x1f\x8b\x08\x00Q_\xb7g\x02\xff\xec\x98i4\xd...,b'\x1f\x8b\x08\x00Q_\xb7g\x02\xff\xec\x97i4\xd...,candid deprecated jd fid expt...,75404,False,2.460718e+06,2,960.0,...,15.229,02143804+5115193,3.060562,32.031876,1.009527,NaN,NaN,NaN,None,NaN
197,WNTR25acewx,b'\x1f\x8b\x08\x00R_\xb7g\x02\xff\xec\x97\xf9W...,b'\x1f\x8b\x08\x00R_\xb7g\x02\xff\xec\x97yTO\x...,b'\x1f\x8b\x08\x00R_\xb7g\x02\xff\xeczi4\x95\x...,candid deprecated jd fid expt...,75405,False,2.460718e+06,2,960.0,...,15.725,02161829+5115316,15.902533,5.649570,1.064954,NaN,NaN,NaN,None,NaN
198,WNTR25acewy,b'\x1f\x8b\x08\x00R_\xb7g\x02\xff\xec\x97i4W\x...,b'\x1f\x8b\x08\x00R_\xb7g\x02\xff\xec\x97y4\x9...,b'\x1f\x8b\x08\x00S_\xb7g\x02\xff\xec\x97y4\xd...,candid deprecated jd fid expt...,75406,False,2.460718e+06,2,960.0,...,15.894,02155798+5115472,18.179668,5.824957,1.067898,66.013519,97.417076,1.137336,None,NaN


In [10]:
res_winter[res_winter["objectid"] == "WNTR25aaexy"]

,objectid,cutout_science,cutout_template,cutout_difference,prv_candidates,candid,deprecated,jd,fid,exptime,...,tmkmag3,tmobjectid3,distgaia,plxgaia,ruwegaia,distgaiabright,plxgaiabright,ruwegaiabright,ztfname,distztf
90,WNTR25aaexy,b'\x1f\x8b\x08\x00~\x82\xb7g\x02\xff\xecyi4\x9...,b'\x1f\x8b\x08\x00~\x82\xb7g\x02\xff\xec\x97\x...,b'\x1f\x8b\x08\x00~\x82\xb7g\x02\xff\xec\x9a\x...,candid progname jd fid isdiff...,85798,False,2.460723e+06,2,960.0,...,13.804,02403115+5418526,1.612213,79.863335,0.972346,50.399048,65.695213,1.025243,ZTF19aaccvvp,1.386896
106,WNTR25aaexy,b'\x1f\x8b\x08\x00\xf7^\xb7g\x02\xff\xec\x9ay4...,b'\x1f\x8b\x08\x00\xf7^\xb7g\x02\xff\xec\x97yT...,b'\x1f\x8b\x08\x00\xf7^\xb7g\x02\xff\xec\x9b\x...,candid progname jd fid isdiff...,76759,False,2.460718e+06,2,960.0,...,15.425,02403199+5419367,0.643009,79.863335,0.972346,51.242401,65.695213,1.025243,ZTF19aaccvvp,0.400611
140,WNTR25aaexy,b'\x1f\x8b\x08\x00]\xab\xb6g\x02\xff\xec\x97\x...,b'\x1f\x8b\x08\x00]\xab\xb6g\x02\xff\xec\x97\x...,b'\x1f\x8b\x08\x00]\xab\xb6g\x02\xff\xecx\xf77...,candid deprecated jd fid expt...,38368,False,2.460715e+06,2,960.0,...,13.804,02403115+5418526,1.028806,79.863335,0.972346,49.933743,65.695213,1.025243,ZTF19aaccvvp,0.593971


In [11]:
lims = []

for t in set(res_winter["jd"]):
    for b in set(res_winter["boardid"]):
        mask = (res_winter["jd"] == t) & ((res_winter["boardid"]) == b)
        match = res_winter[mask]
        if len(match) > 0:
            lims.append(match["diffmaglim"].iloc[0])

print(f"Found avros from {len(lims)} unique difference images, with median depth of {np.median(lims):.1f} AB mag")

Found avros from 305 unique difference images, with median depth of 17.8 AB mag


In [12]:
with open(ztf_initial_cache, "r") as f:
    res_ztf = json.load(f)
    
ztf_sources = []

for entry in res_ztf:

    new_dict = {}
    
    for field in ["candid", "objectId"]:
        new_dict[field] = entry[field]

    for key, val in entry["candidate"].items():
        new_dict[key] = val

    ztf_sources.append(new_dict)

ztf_df = pd.DataFrame(ztf_sources)
ztf_df

,candid,objectId,jd,fid,pid,diffmaglim,pdiffimfilename,programpi,programid,isdiffpos,...,zpmed,clrmed,clrrms,neargaia,neargaiabright,maggaia,maggaiabright,exptime,drb,drbversion
0,2960141290115010001,ZTF24aaqpnqr,2.460715e+06,2,2960141290115,20.767807,ztf_20250208141285_000738_zr_c01_o_q2_scimrefd...,Kulkarni,2,f,...,28.773001,0.499,0.221884,0.575656,-999.000000,17.355162,-999.000000,299.0,0.999831,d6_m7
1,2960141290115010005,ZTF23abclilr,2.460715e+06,2,2960141290115,20.767807,ztf_20250208141285_000738_zr_c01_o_q2_scimrefd...,Kulkarni,2,f,...,28.773001,0.499,0.221884,0.403519,-999.000000,17.613058,-999.000000,299.0,0.999991,d6_m7
2,2960141290115015012,ZTF18aabfvwb,2.460715e+06,2,2960141290115,20.767807,ztf_20250208141285_000738_zr_c01_o_q2_scimrefd...,Kulkarni,2,t,...,28.773001,0.499,0.221884,0.187413,66.061272,16.920624,13.701635,299.0,1.000000,d6_m7
3,2960141290115010004,ZTF18abpxsas,2.460715e+06,2,2960141290115,20.767807,ztf_20250208141285_000738_zr_c01_o_q2_scimrefd...,Kulkarni,2,f,...,28.773001,0.499,0.221884,0.773030,50.828609,15.883644,10.136169,299.0,0.577578,d6_m7
4,2960141290115010010,ZTF18abosmjn,2.460715e+06,2,2960141290115,20.767807,ztf_20250208141285_000738_zr_c01_o_q2_scimrefd...,Kulkarni,2,f,...,28.773001,0.499,0.221884,0.361587,22.573967,16.025274,12.531811,299.0,0.999758,d6_m7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116156,2963397844015015000,ZTF20acwyokj,2.460718e+06,1,2963397844015,18.400349,ztf_20250211397847_000266_zg_c11_o_q1_scimrefd...,Qin,3,t,...,26.122000,0.493,0.181838,38.648453,-999.000000,19.225090,-999.000000,30.0,0.008988,d6_m7
116157,2969420654015015000,ZTF20acwyokj,2.460724e+06,1,2969420654015,18.792280,ztf_20250217420660_000266_zg_c11_o_q1_scimrefd...,Kulkarni,1,t,...,25.931999,0.502,0.208582,39.542866,-999.000000,19.225090,-999.000000,30.0,0.057140,d6_m7
116158,2970398734015015000,ZTF20acwyokj,2.460725e+06,1,2970398734015,18.950897,ztf_20250218398738_000266_zg_c11_o_q1_scimrefd...,Kulkarni,1,t,...,25.877001,0.505,0.210025,39.144318,-999.000000,19.225090,-999.000000,30.0,0.967589,d6_m7
116159,2971376084015015000,ZTF20acwyokj,2.460726e+06,1,2971376084015,18.984457,ztf_20250219376076_000266_zg_c11_o_q1_scimrefd...,Kulkarni,1,t,...,25.903999,0.500,0.204263,39.106178,-999.000000,19.225090,-999.000000,30.0,0.968087,d6_m7


In [13]:
ztf_df

,candid,objectId,jd,fid,pid,diffmaglim,pdiffimfilename,programpi,programid,isdiffpos,...,zpmed,clrmed,clrrms,neargaia,neargaiabright,maggaia,maggaiabright,exptime,drb,drbversion
0,2960141290115010001,ZTF24aaqpnqr,2.460715e+06,2,2960141290115,20.767807,ztf_20250208141285_000738_zr_c01_o_q2_scimrefd...,Kulkarni,2,f,...,28.773001,0.499,0.221884,0.575656,-999.000000,17.355162,-999.000000,299.0,0.999831,d6_m7
1,2960141290115010005,ZTF23abclilr,2.460715e+06,2,2960141290115,20.767807,ztf_20250208141285_000738_zr_c01_o_q2_scimrefd...,Kulkarni,2,f,...,28.773001,0.499,0.221884,0.403519,-999.000000,17.613058,-999.000000,299.0,0.999991,d6_m7
2,2960141290115015012,ZTF18aabfvwb,2.460715e+06,2,2960141290115,20.767807,ztf_20250208141285_000738_zr_c01_o_q2_scimrefd...,Kulkarni,2,t,...,28.773001,0.499,0.221884,0.187413,66.061272,16.920624,13.701635,299.0,1.000000,d6_m7
3,2960141290115010004,ZTF18abpxsas,2.460715e+06,2,2960141290115,20.767807,ztf_20250208141285_000738_zr_c01_o_q2_scimrefd...,Kulkarni,2,f,...,28.773001,0.499,0.221884,0.773030,50.828609,15.883644,10.136169,299.0,0.577578,d6_m7
4,2960141290115010010,ZTF18abosmjn,2.460715e+06,2,2960141290115,20.767807,ztf_20250208141285_000738_zr_c01_o_q2_scimrefd...,Kulkarni,2,f,...,28.773001,0.499,0.221884,0.361587,22.573967,16.025274,12.531811,299.0,0.999758,d6_m7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116156,2963397844015015000,ZTF20acwyokj,2.460718e+06,1,2963397844015,18.400349,ztf_20250211397847_000266_zg_c11_o_q1_scimrefd...,Qin,3,t,...,26.122000,0.493,0.181838,38.648453,-999.000000,19.225090,-999.000000,30.0,0.008988,d6_m7
116157,2969420654015015000,ZTF20acwyokj,2.460724e+06,1,2969420654015,18.792280,ztf_20250217420660_000266_zg_c11_o_q1_scimrefd...,Kulkarni,1,t,...,25.931999,0.502,0.208582,39.542866,-999.000000,19.225090,-999.000000,30.0,0.057140,d6_m7
116158,2970398734015015000,ZTF20acwyokj,2.460725e+06,1,2970398734015,18.950897,ztf_20250218398738_000266_zg_c11_o_q1_scimrefd...,Kulkarni,1,t,...,25.877001,0.505,0.210025,39.144318,-999.000000,19.225090,-999.000000,30.0,0.967589,d6_m7
116159,2971376084015015000,ZTF20acwyokj,2.460726e+06,1,2971376084015,18.984457,ztf_20250219376076_000266_zg_c11_o_q1_scimrefd...,Kulkarni,1,t,...,25.903999,0.500,0.204263,39.106178,-999.000000,19.225090,-999.000000,30.0,0.968087,d6_m7


In [14]:
sgscore_cut = 0.5
dist_cut_arcsec = 3.

def stellar_cut(df):
    print("Applying stellar cut")
    # star_cut = (pd.notnull(df["distpsnr1"]) & (df["distpsnr1"] < dist_cut_arcsec) & pd.notnull(df["sgscore1"]) & (df["sgscore1"] > sgscore_cut))

    star_cut = (  # Not a star according to PS1 or PS1STRM
        (
            (df["distpsnr1"] < 7.0)
            & (df["srmag1"] < 15)
            & ((df["sgscore1"] > sgscore_cut))# | (df["ps1strmprobstar1"] > 0.7))
        )
        | (
            (df["distpsnr1"] < dist_cut_arcsec)
            # & (df["srmag1"] < 18)
            & ((df["sgscore1"] > sgscore_cut))# | (df["ps1strmprobstar1"] > 0.7))
        )
        | (
            (df["distgaia"] < dist_cut_arcsec)
            # & (df["srmag1"] < 18)
            & ((df["plxgaia"] > 3.0))
        )
        | (
            (df["distgaiabright"] < 20.)
            # & (df["srmag1"] < 18)
            # & ((df["plxgaia"] > 3.0))
        )
            
    )
    
    print(f"{np.sum(~star_cut)} / {len(star_cut)} alerts pass star cuts")
    new = df[~star_cut]
    print(f"{len(set(new['objectid']))} unique sources")
    return new

In [15]:
def time_cut(df, t_min_jd):

    mask = (df["jdstarthist"] < t_min_jd)
    print(f"Rejecting {mask.sum()} sources with WINTER predections")
    df = df[~mask]

    ztf_mask = pd.notnull(df[ZTF_NAME_KEY]).astype(bool).to_numpy()
    matches = df[ztf_mask]

    if ztf_mask.sum() > 0:
        has_pre = []
        for _, row in matches.iterrows():
            has_pre.append(row[ZTF_HIST_KEY]["jdstarthist"].min() < t_min_jd)
        has_pre = np.array(has_pre, dtype=bool)
        print(f"Rejecting {has_pre.sum()} sources with ZTF predections")
        ztf_mask[ztf_mask] *= has_pre
        df = df[~ztf_mask]
    return df
    

In [16]:
def apply_cuts(base_df, t_min_jd=None):
    df = base_df.copy()
    print(f"Deduplicating {len(df)} alerts")
    df = deduplicate_df(df)
    print(f"Have {len(df)} sources")
    df = stellar_cut(df)
    df = add_ztf_candidates(df)
    if t_min_jd is not None:
        df = time_cut(df, t_min_jd)
    return df

In [17]:
mask = pd.notnull(res_winter[ZTF_NAME_KEY])
print(f"A total of {mask.sum()} have ZTF crossmatches already")
print(f"This corresponds to {len(set(res_winter['objectid'][mask]))} sources")

A total of 11873 have ZTF crossmatches already
This corresponds to 9608 sources


In [18]:
ztf_radius = 3.0

if len(ztf_df) > 0:
    ztf_positions = SkyCoord(ztf_df["ra"].to_numpy()*u.deg, ztf_df["dec"].to_numpy()*u.deg, frame='icrs')
    
    winter_positions = SkyCoord(res_winter["ra"].to_numpy()*u.deg, res_winter["dec"].to_numpy()*u.deg, frame='icrs')
    idx, d2d, _ = winter_positions.match_to_catalog_sky(ztf_positions)
    
    crossmatch_radius = ztf_radius * u.arcsec
    mask = d2d < crossmatch_radius

    ztf_names = []

    for i in idx[mask]:
        ztf_names.append(ztf_df["objectId"].iloc[i])

    dists = [x.arcsec for x in d2d[mask]]

    print(f"Found {len(dists)} matches")

    matchmask = pd.isnull(res_winter[ZTF_NAME_KEY][mask])
    
    res_winter.loc[mask, [ZTF_NAME_KEY]] = ztf_names
    res_winter.loc[mask, ["distztf"]] = dists

Found 1275 matches


In [19]:
ztf_mask = pd.notnull(res_winter[ZTF_NAME_KEY])
print(f"A total of {ztf_mask.sum()} have ZTF crossmatches already")
print(f"This corresponds to {len(set(res_winter['objectid'][ztf_mask]))} sources")
cut_ztf = apply_cuts(res_winter[ztf_mask], t_min_jd=(nu.t_min.jd if not neutrino else None))

A total of 11947 have ZTF crossmatches already
This corresponds to 9673 sources
Deduplicating 11947 alerts
Have 9673 sources
Applying stellar cut
140 / 9673 alerts pass star cuts
140 unique sources


100%|█████████████████████████████████████████| 140/140 [00:19<00:00,  7.04it/s]

Rejecting 0 sources with WINTER predections
Rejecting 136 sources with ZTF predections


In [20]:
if len(cut_ztf) > 0:
    outpath = base_outpath.replace("combined", "combined_ztf")
    
    print(f"Saving PDF to {outpath}")
    
    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut_ztf.iterrows(), total=len(cut_ztf)):
            generate_single_page(row, ann_fields=ann_fields)
            pdf.savefig()
            plt.close()

    if crossmatch:

        outpath = base_outpath.replace("combined", "combined_ztf_crossmatched")
    
        with PdfPages(outpath) as pdf:
            for i, row in tqdm(cut_ztf.iterrows(), total=len(cut_ztf)):
                generate_single_page(row, ann_fields=ann_fields, crossmatch=True)
                pdf.savefig()
                plt.close()
else:
    print("No WINTER+ZTF candidates found")

Saving PDF to /Users/rdstein/Data/nuwinter/S250206dm/winter_candidate_pdf/combined_ztf.pdf


100%|█████████████████████████████████████████████| 4/4 [00:00<00:00, 28.22it/s]


In [21]:
masks = [
    res_winter["ndethist"] > 0,
]

winter_mask = np.ones(len(res_winter), dtype=bool)
for new_mask in masks:
    winter_mask *= new_mask
    
print(f"{np.sum(mask)} / {len(mask)} alerts pass all WINTER-WINTER cuts")
cut_winter = res_winter[winter_mask]
print(f"{len(set(cut_winter['objectid']))} unique sources")

cut_winter = apply_cuts(cut_winter, t_min_jd=(nu.t_min.jd if not neutrino else None))


if len(cut_winter) > 0:
    cut_winter.sort_values(by="objectid", inplace=True)
    outpath = base_outpath.replace("combined", "combined_2dets")
    print(f"Saving PDF to {outpath}")
    
    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut_winter.iterrows(), total=len(cut_winter)):
            generate_single_page(row, ann_fields=ann_fields, crossmatch=False)
            pdf.savefig()
            plt.close()

    if crossmatch:
        outpath = base_outpath.replace("combined", "combined_2dets_crossmatched")
        print(f"Saving PDF to {outpath}")
        
        with PdfPages(outpath) as pdf:
            for i, row in tqdm(cut_winter.iterrows(), total=len(cut_winter)):
                generate_single_page(row, ann_fields=ann_fields, crossmatch=True)
                pdf.savefig()
                plt.close()

1275 / 62675 alerts pass all WINTER-WINTER cuts
4141 unique sources
Deduplicating 5176 alerts
Have 4141 sources
Applying stellar cut
140 / 4141 alerts pass star cuts
140 unique sources


100%|████████████████████████████████████████| 140/140 [00:00<00:00, 154.26it/s]


Rejecting 0 sources with WINTER predections
Rejecting 5 sources with ZTF predections
Saving PDF to /Users/rdstein/Data/nuwinter/S250206dm/winter_candidate_pdf/combined_2dets.pdf


100%|█████████████████████████████████████████| 135/135 [00:04<00:00, 27.11it/s]


Saving PDF to /Users/rdstein/Data/nuwinter/S250206dm/winter_candidate_pdf/combined_2dets_crossmatched.pdf


100%|█████████████████████████████████████████| 135/135 [00:05<00:00, 25.46it/s]


In [22]:
# Download the TNS after JSON for GW event, 

tns_dir = get_pdf_path(name).parent.parent

print(f"Looking for TNS json in {tns_dir}")

tns_blob = list(tns_dir.glob("*.json"))

tns_df = pd.read_json(tns_blob[0]).transpose() if len(tns_blob) > 0 else pd.DataFrame()
tns_df

Looking for TNS json in /Users/rdstein/Data/nuwinter/S250206dm


,name_prefix,name,ra,declination,type,source_group,discoverymag,filter,discoverydate,credibility_level,within_region
2025bnx,AT,2025bnx,243.1983388877001,-68.82776127375323,None,GW-MMADS,20.5293,r,2025-02-13 07:39:14.688,0.447,50.0%
2025bno,AT,2025bno,242.69847376101356,-68.4704449089745,None,GW-MMADS,21.1144,i,2025-02-13 06:38:00.096,0.488,50.0%
2025bnm,AT,2025bnm,245.7173645297777,-69.0233069359075,None,GW-MMADS,22.1041,i,2025-02-13 06:46:37.632,0.46,50.0%
2025bnl,AT,2025bnl,242.33079763537705,-69.34194454572027,None,GW-MMADS,21.3403,i,2025-02-13 06:38:00.096,0.449,50.0%
2025bnh,AT,2025bnh,248.1591995919586,-68.51748068351628,None,GW-MMADS,21.4456,i,2025-02-13 07:06:47.232,0.497,50.0%
...,...,...,...,...,...,...,...,...,...,...,...
2025bai,AT,2025bai,151.935085324,-20.3429319805,None,Pan-STARRS,19.78,r,2025-02-07 10:16:09.696,0.935,99.0%
2025bah,AT,2025bah,152.779895922,-21.3745584038,Galaxy,Pan-STARRS,18.69,r,2025-02-07 10:08:38.688,0.92,99.0%
2025baf,AT,2025baf,158.506624546,-29.7469922495,None,Pan-STARRS,19.33,r,2025-02-07 09:48:18.720,0.947,99.0%
2025bar,SN,2025bar,170.681922,-45.574606,SN Ia,GOTO,20.23,L,2025-02-07 13:52:45.120,0.945,99.0%


In [23]:
tns_mask = np.zeros(len(res_winter))
if len(tns_df) > 0:
    tns_positions = SkyCoord(tns_df["ra"].to_numpy(dtype=float)*u.deg, tns_df["declination"].to_numpy(dtype=float)*u.deg, frame='icrs')
    
    winter_positions = SkyCoord(res_winter["ra"].to_numpy()*u.deg, res_winter["dec"].to_numpy()*u.deg, frame='icrs')
    idx, d2d, _ = winter_positions.match_to_catalog_sky(tns_positions)
    
    crossmatch_radius = 3.0 * u.arcsec
    tns_mask = d2d < crossmatch_radius

    print(f"Found {mask.sum()}/{len(winter_positions)} matches to {len(tns_df)} TNS sources.")

Found 1275/62675 matches to 116 TNS sources.


In [24]:
cut_tns = res_winter[tns_mask]
cut_tns

,objectid,cutout_science,cutout_template,cutout_difference,prv_candidates,candid,deprecated,jd,fid,exptime,...,tmkmag3,tmobjectid3,distgaia,plxgaia,ruwegaia,distgaiabright,plxgaiabright,ruwegaiabright,ztfname,distztf


In [25]:
if len(cut_tns) > 0:
    outpath = base_outpath.replace("combined", "combined_tns")
    
    print(f"Saving PDF to {outpath}")
    
    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut_tns.iterrows(), total=len(cut_tns)):
            generate_single_page(row, ann_fields=ann_fields)
            pdf.savefig()
            plt.close()

    if crossmatch:

        outpath = base_outpath.replace("combined", "combined_tns_crossmatched")
    
        with PdfPages(outpath) as pdf:
            for i, row in tqdm(cut_tns.iterrows(), total=len(cut_tns)):
                generate_single_page(row, ann_fields=ann_fields, crossmatch=True)
                pdf.savefig()
                plt.close()
else:
    print("No TNS candidates found")

No TNS candidates found


In [26]:
mask_any = tns_mask | ztf_mask | winter_mask
cut_any = res_winter[mask_any]

In [27]:
cut_any

,objectid,cutout_science,cutout_template,cutout_difference,prv_candidates,candid,deprecated,jd,fid,exptime,...,tmkmag3,tmobjectid3,distgaia,plxgaia,ruwegaia,distgaiabright,plxgaiabright,ruwegaiabright,ztfname,distztf
1,WNTR25aacyt,b'\x1f\x8b\x08\x00\xd1\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd1\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd1\xaa\xb6g\x02\xff\xec\x9...,candid deprecated jd fid expt...,37037,False,2.460715e+06,2,960.0,...,13.901000,02455735+5448057,0.769464,17.789389,1.012096,17.490801,64.846901,0.981179,ZTF18acmgeko,0.231566
6,WNTR25aacyy,b'\x1f\x8b\x08\x00\xd5\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd5\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd5\xaa\xb6g\x02\xff\xec\x9...,candid deprecated jd fid expt...,37042,False,2.460715e+06,2,960.0,...,14.021000,02435896+5446473,0.559391,138.421722,3.757299,0.559391,138.421722,3.757299,ZTF18abxtdoz,0.112234
7,WNTR25aacyz,b'\x1f\x8b\x08\x00\xd5\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd5\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd5\xaa\xb6g\x02\xff\xec\xb...,candid deprecated jd fid expt...,37043,False,2.460715e+06,2,960.0,...,15.315000,02452293+5446451,0.774077,22.747139,1.022857,55.369640,33.908730,2.294901,ZTF19abtahvb,0.637266
11,WNTR25aaczd,b'\x1f\x8b\x08\x00\xd8\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd8\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd8\xaa\xb6g\x02\xff\xec\x9...,candid deprecated jd fid expt...,37047,False,2.460715e+06,2,960.0,...,14.576000,02450803+5446082,1.558647,8.516641,1.091765,62.614559,68.093529,1.048480,ZTF20abihmcy,1.460376
12,WNTR25aacze,b'\x1f\x8b\x08\x00\xd9\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd9\xaa\xb6g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xd9\xaa\xb6g\x02\xff\xec\x9...,candid deprecated jd fid expt...,37048,False,2.460715e+06,2,960.0,...,15.310000,02455743+5445512,1.419714,67.285393,1.010768,36.592575,24.046335,0.987211,ZTF21abhxqnh,0.994473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,WNTR25acevy,b'\x1f\x8b\x08\x00=_\xb7g\x02\xff\xec\x97\xf97...,b'\x1f\x8b\x08\x00=_\xb7g\x02\xff\xec\x97\xf97...,b'\x1f\x8b\x08\x00=_\xb7g\x02\xff\xec\x97i4\x9...,candid deprecated jd fid expt...,75379,False,2.460718e+06,2,960.0,...,14.812000,02160428+5117552,0.681225,32.692684,0.967254,55.908997,56.793720,1.501800,ZTF18achmyzj,0.459149
172,WNTR25acevz,b'\x1f\x8b\x08\x00>_\xb7g\x02\xff\xec\x9a\xe5S...,b'\x1f\x8b\x08\x00>_\xb7g\x02\xff\xec\x97y0\xd...,b'\x1f\x8b\x08\x00>_\xb7g\x02\xff\xec\xb9y4\x9...,candid deprecated jd fid expt...,75380,False,2.460718e+06,2,960.0,...,15.770000,02161406+5117305,4.001687,56.793720,1.501800,4.001687,56.793720,1.501800,ZTF19acxmict,0.683792
176,WNTR25abgmz,b'\x1f\x8b\x08\x00A_\xb7g\x02\xff\xec\x97i4\x9...,b'\x1f\x8b\x08\x00A_\xb7g\x02\xff\xec\x97y4\x9...,b'\x1f\x8b\x08\x00A_\xb7g\x02\xff\xec\x97y4\xd...,candid progname jd fid isdiff...,75384,False,2.460718e+06,2,960.0,...,NaN,nan,0.309921,0.899485,1.016951,83.080666,69.500191,1.054173,None,NaN
179,WNTR25acewf,b'\x1f\x8b\x08\x00B_\xb7g\x02\xff\xec\x97i4\x9...,b'\x1f\x8b\x08\x00B_\xb7g\x02\xff\xec\x97\xf97...,b'\x1f\x8b\x08\x00B_\xb7g\x02\xff\xecw\xf97\xd...,candid deprecated jd fid expt...,75387,False,2.460718e+06,2,960.0,...,16.504999,02154344+5116272,3.400429,199.202682,1.571166,38.054306,49.287838,1.276158,ZTF20aamipjn,1.982602


In [28]:
cut_any = apply_cuts(cut_any, t_min_jd=(nu.t_min.jd if not neutrino else None))


if len(cut_any) > 0:
    cut_any.sort_values(by="objectid", inplace=True)
    outpath = base_outpath.replace("combined", "combined_any")
    print(f"Saving PDF to {outpath}")
    
    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut_any.iterrows(), total=len(cut_any)):
            generate_single_page(row, ann_fields=ann_fields, crossmatch=False)
            pdf.savefig()
            plt.close()

    if crossmatch:
        outpath = base_outpath.replace("combined", "combined_any_crossmatched")
        print(f"Saving PDF to {outpath}")
        
        with PdfPages(outpath) as pdf:
            for i, row in tqdm(cut_any.iterrows(), total=len(cut_any)):
                generate_single_page(row, ann_fields=ann_fields, crossmatch=True)
                pdf.savefig()
                plt.close()

Deduplicating 14679 alerts
Have 11802 sources
Applying stellar cut
274 / 11802 alerts pass star cuts
274 unique sources


100%|█████████████████████████████████████████| 274/274 [00:19<00:00, 14.36it/s]


Rejecting 0 sources with WINTER predections
Rejecting 135 sources with ZTF predections
Saving PDF to /Users/rdstein/Data/nuwinter/S250206dm/winter_candidate_pdf/combined_any.pdf


100%|█████████████████████████████████████████| 139/139 [00:05<00:00, 24.90it/s]


Saving PDF to /Users/rdstein/Data/nuwinter/S250206dm/winter_candidate_pdf/combined_any_crossmatched.pdf


100%|█████████████████████████████████████████| 139/139 [00:05<00:00, 26.75it/s]


In [29]:
cut_any

,objectid,cutout_science,cutout_template,cutout_difference,prv_candidates,candid,deprecated,jd,fid,exptime,...,tmobjectid3,distgaia,plxgaia,ruwegaia,distgaiabright,plxgaiabright,ruwegaiabright,ztfname,distztf,ztf_candidates
6678,WNTR25aaahn,b'\x1f\x8b\x08\x00q\x82\xb7g\x02\xff\xec\x99i4...,b'\x1f\x8b\x08\x00q\x82\xb7g\x02\xff\xecwi0\xd...,b'\x1f\x8b\x08\x00q\x82\xb7g\x02\xff\xec\xdbgP...,candid progname jd fid isdiff...,85924,False,2460722.791569,2,960.0,...,02323068+5418486,8.856771,0.397649,1.078827,29.110882,35.403782,1.03817,None,NaN,None
10851,WNTR25aabet,b'\x1f\x8b\x08\x001\xa8\xb7g\x02\xff\xec\x97i4...,b'\x1f\x8b\x08\x001\xa8\xb7g\x02\xff\xec\x97iT...,b'\x1f\x8b\x08\x001\xa8\xb7g\x02\xff\xec\x9bi4...,candid progname jd fid isdiff...,97495,False,2460723.689046,2,960.0,...,02162769+5236543,11.298206,0.999484,0.960546,NaN,NaN,NaN,None,NaN,None
3469,WNTR25aabma,b'\x1f\x8b\x08\x00\xb1_\xb7g\x02\xff\xec\x97\x...,b'\x1f\x8b\x08\x00\xb1_\xb7g\x02\xff\xec\x97i4...,b'\x1f\x8b\x08\x00\xb1_\xb7g\x02\xff\xec{\xf97...,candid progname jd fid isdiff...,83773,False,2460717.81892,2,960.0,...,nan,7.413851,1.45726,0.982674,43.655209,97.561157,0.985429,None,NaN,None
8490,WNTR25aabna,b'\x1f\x8b\x08\x00\x88\x82\xb7g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\x88\x82\xb7g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\x88\x82\xb7g\x02\xff\xecwi4...,candid progname jd fid isdiff...,84962,False,2460722.815649,2,960.0,...,nan,11.721796,NaN,NaN,51.959164,97.561157,0.985429,None,NaN,None
7794,WNTR25aabpy,b'\x1f\x8b\x08\x00h\x82\xb7g\x02\xff\xec\x97y4...,b'\x1f\x8b\x08\x00h\x82\xb7g\x02\xff\xec\x98\x...,b'\x1f\x8b\x08\x00h\x82\xb7g\x02\xff\xecwi4\x9...,candid progname jd fid isdiff...,84067,False,2460722.755559,2,960.0,...,02324939+5243437,4.626829,250.067612,0.892434,51.154049,24.976233,0.985508,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9590,WNTR25acpsl,b'\x1f\x8b\x08\x00\xa2\x82\xb7g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xa2\x82\xb7g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xa2\x82\xb7g\x02\xff\xec\x9...,candid progname jd fid isdiff...,87904,False,2460722.755559,2,960.0,...,02354973+5237364,12.223631,0.995101,0.990333,NaN,NaN,NaN,None,NaN,None
4096,WNTR25acqis,b'\x1f\x8b\x08\x00j\x82\xb7g\x02\xff\xec\x97\x...,b'\x1f\x8b\x08\x00j\x82\xb7g\x02\xff\xec\x97\x...,b'\x1f\x8b\x08\x00j\x82\xb7g\x02\xff\xecwi4\xd...,candid deprecated jd fid expt...,84249,False,2460722.767579,2,960.0,...,nan,3.349981,253.271347,1.087381,54.829567,33.763939,1.001381,ZTF25aafiyed,1.322718,jd fid pid diffmagli...
2604,WNTR25acqzn,b'\x1f\x8b\x08\x00X\x82\xb7g\x02\xff\xec\x97iT...,b'\x1f\x8b\x08\x00X\x82\xb7g\x02\xff\xed\xd7i4...,b'\x1f\x8b\x08\x00X\x82\xb7g\x02\xff\xec\x99i4...,candid progname jd fid isdiff...,84862,False,2460722.815649,2,960.0,...,02403506+5344105,4.714442,23.815172,0.973108,44.623627,122.81871,0.967252,None,NaN,None
4679,WNTR25acttk,b'\x1f\x8b\x08\x00\xc7\xa7\xb7g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xc7\xa7\xb7g\x02\xff\xec\x9...,b'\x1f\x8b\x08\x00\xc7\xa7\xb7g\x02\xff\xec\x9...,candid progname jd fid isdiff...,91256,False,2460723.725172,2,960.0,...,nan,14.83335,NaN,NaN,81.22113,32.522511,1.064975,None,NaN,None


In [30]:
all_names = cut_any["objectid"].tolist()
sep_names = cut_ztf["objectid"].tolist() + cut_winter["objectid"].tolist()

In [31]:
[x for x in sep_names if x not in all_names]

[]